In [18]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()


In [23]:
import asyncpg
import numpy as np
import matplotlib.pyplot as plt

async def fetch_pheromone_matrix_numpy(pool, episode_id: int, timestep: int, grid_size: int) -> np.ndarray:
    query = """
    SELECT pheromone
    FROM pheromone_matrix_logs
    WHERE episode_id = $1 AND timestep = $2;
    """
    row = await pool.fetchrow(query, episode_id, timestep)
    if row is None:
        raise ValueError("No pheromone matrix found for the given episode and timestep.")

    byte_data = row['pheromone']
    shape = (grid_size + 2, grid_size + 2)
    return np.frombuffer(byte_data, dtype=np.float64).reshape(shape)

async def run_main():
    pool = await asyncpg.create_pool(
        user='postgres',
        password='1',
        database='webots_drone_async',
        host='localhost'
    )

    grid_size = 500  # change to your GRID_SIZE
    matrix = await fetch_pheromone_matrix_numpy(pool, episode_id=20, timestep=1653, grid_size=grid_size)

    await pool.close()
    return matrix


In [24]:
pheromone_matrix = await run_main()


In [26]:
pheromone_matrix[240:260, 240:260]

array([[0.91391537, 0.91104327, 0.90905214, 0.90782519, 0.90713144,
        0.90676018, 0.90656776, 0.90646975, 0.90642028, 0.90639543,
        0.90641291, 0.90645141, 0.90652962, 0.90668448, 0.90698565,
        0.90755291, 0.90856695, 0.91022848, 0.91262536, 0.91558666],
       [0.91012873, 0.90581688, 0.90237192, 0.89997162, 0.89848758,
        0.89764678, 0.89719633, 0.89696272, 0.8968437 , 0.89678361,
        0.89682588, 0.89691856, 0.89710535, 0.89746948, 0.89815692,
        0.89938388, 0.90138899, 0.90426987, 0.90780996, 0.91155518],
       [0.90699554, 0.9012689 , 0.89604866, 0.89184533, 0.88889944,
        0.88707097, 0.88603259, 0.88547555, 0.88518646, 0.88503912,
        0.88514274, 0.88536808, 0.88581516, 0.8866608 , 0.88817267,
        0.89063605, 0.89415922, 0.89846206, 0.90298178, 0.90724245],
       [0.90447309, 0.89747749, 0.89048898, 0.88405013, 0.87881896,
        0.87512801, 0.87282673, 0.87151624, 0.87081213, 0.87044642,
        0.87070347, 0.87125337, 0.87231189, 0

In [31]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_PHEROMONE = torch.exp2(torch.tensor(100, dtype=torch.float64, device=device))
GRID_SIZE = torch.tensor( 500, device=device)  # 500x500 grid
PHEROMONE_MATRIX = torch.zeros((GRID_SIZE+2, GRID_SIZE+2), dtype=torch.float64, device=device)
    # Generate row and column indices with float64 precision
i_indices = torch.arange(GRID_SIZE+2, dtype=torch.float64, device=device)  # Row indices
j_indices = torch.arange(GRID_SIZE+2, dtype=torch.float64, device=device)  # Column indices
    # Create a meshgrid of indices for rows and columns
i_grid, j_grid = torch.meshgrid(i_indices, j_indices, indexing='ij')

new_pheromone_matrix = torch.zeros_like(PHEROMONE_MATRIX, dtype=torch.float64, device=device)


In [46]:
row=252
col=250

In [47]:
new_pheromone_matrix+= MAX_PHEROMONE / (1+torch.exp2(torch.abs(row - i_grid)) + torch.exp2(torch.abs(col - j_grid)))


In [48]:
new_pheromone_matrix[251, 251]

tensor(5.0706e+29, device='cuda:0', dtype=torch.float64)

In [49]:
PHEROMONE_MATRIX = (PHEROMONE_MATRIX  + new_pheromone_matrix)/2
PHEROMONE_MATRIX = torch.clip(PHEROMONE_MATRIX, 0, MAX_PHEROMONE)

In [52]:
PHEROMONE_MATRIX[252, 250]

tensor(3.1691e+29, device='cuda:0', dtype=torch.float64)

In [7]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
GRID_SIZE = torch.tensor( 500, device=device)  # 500x500 grid
FLOOR_WIDTH = torch.tensor(835, device=device)
FLOOR_DEPTH = torch.tensor(913, device=device)
CELL_WIDTH = FLOOR_WIDTH / GRID_SIZE
CELL_DEPTH = FLOOR_DEPTH / GRID_SIZE
MAX_TARGET_DISTANCE = torch.sqrt(CELL_WIDTH**2 + CELL_DEPTH**2)
GRID_BORDERS=torch.zeros((GRID_SIZE + 2, GRID_SIZE + 2, 6), dtype=torch.float64, device=device)
MAX_PHEROMONE = torch.exp2(torch.tensor(20, dtype=torch.float64, device=device))
def init_grid_borders():
    global GRID_BORDERS
    # Create a tensor for grid borders
    half_cell_width = CELL_WIDTH / 2
    half_cell_height = CELL_DEPTH / 2

    # Create the grid indices
    i_indices = torch.arange(GRID_SIZE.item() + 2, device=device).float()
    j_indices = torch.arange(GRID_SIZE.item() + 2, device=device).float()

    # Create meshgrid for i and j indices
    i_grid, j_grid = torch.meshgrid(i_indices, j_indices, indexing='ij')

    # Calculate the X and Y values for the grid
    top_y = -FLOOR_DEPTH / 2 + i_grid*CELL_DEPTH
    right_x = -FLOOR_WIDTH / 2 + j_grid * CELL_WIDTH  # right_x = -FLOOR_WIDTH/2 + j * CELL_WIDTH
    middle_x = right_x - half_cell_width
    left_x = right_x - CELL_WIDTH
    middle_y = top_y - half_cell_height
    bottom_y = top_y - CELL_DEPTH


    # Populate the GRID_BORDERS tensor with vectorized assignment
    GRID_BORDERS[..., 0] = left_x  # left_x
    GRID_BORDERS[..., 1] = middle_x  # middle_x
    GRID_BORDERS[..., 2] = right_x  # right_x
    GRID_BORDERS[..., 3] = bottom_y  # bottom_y
    GRID_BORDERS[..., 4] = middle_y  # middle_y
    GRID_BORDERS[..., 5] = top_y  # top_y

    # Optionally, move the result to GPU if needed
    GRID_BORDERS = GRID_BORDERS.to(device)

init_grid_borders()
def find_grid_coordinates(pos):
    """
    Find the grid cell (row, column) corresponding to the given (x, y) world coordinates.
    Grid starts from 1.
    """
    column = int(torch.floor((pos[0] + FLOOR_WIDTH / 2) / CELL_WIDTH)) + 1
    row = int(torch.floor((pos[1] + FLOOR_DEPTH / 2) / CELL_DEPTH)) + 1
    left_x, middle_x, right_x, bottom_y, middle_y, top_y=grid_borders(row, column)
    if pos[0] < left_x:
        column = column - 1
    if pos[1] < bottom_y:
        row = row - 1
    if pos[0] > right_x:
        column = column + 1
    if pos[1] > top_y:
        row = row + 1

    if not 0 <= row <= GRID_SIZE+1 or not 0 <= column <= GRID_SIZE+1:
        print(f"Coordinates ({pos}) are out of grid bounds!")
    return torch.tensor([row, column], dtype=torch.int, device=device)

def grid_borders(row, column):
    """
    Given a grid coordinate (row, column), calculate the world-coordinate borders of that grid cell.
    """
    # x_min = (column - 1) * CELL_WIDTH - FLOOR_WIDTH / 2
    # y_min = (row - 1) * CELL_DEPTH - FLOOR_DEPTH / 2
    # x_max = column * CELL_WIDTH - FLOOR_WIDTH / 2
    # y_max = row * CELL_DEPTH - FLOOR_DEPTH / 2

    # return x_min, y_min, x_max, y_max
    return GRID_BORDERS[row, column]
def angle_between_vectors(v1, v2):
    """Calculate the signed angle between two 2D vectors using PyTorch."""
    dot = torch.dot(v1, v2)  # Dot product
    det = v1[0] * v2[1] - v1[1] * v2[0]  # Determinant (cross product in 2D)
    angle = torch.atan2(det, dot)  # Signed angle in radians
    return angle


Using device: cuda


In [8]:
pos=torch.tensor([100,100], dtype=torch.float64, device=device)

In [9]:
row, col =find_grid_coordinates(pos)

In [10]:
grid_borders(row, col)

tensor([ 98.5300,  99.3650, 100.2000,  98.6040,  99.5170, 100.4300],
       device='cuda:0', dtype=torch.float64)